# 3. Grouping with Pivot Tables

### Summary

* Pivot tables perform the same operation as a groupby, but display the data in wide rather than long format
* Pivot tables can make comparisons between groups easier

## Pivot Table Definition
Take a look at the following diagram of a pivot table from Microsoft Excel. A pivot table aggregates data, usually between the intersection of two columns of your data. In the pivot table below, the two dimensions are the `Region` and the `Qtr`. All pivot tables must aggregate some other column of data. Here, it is the revenue and it is summed.

There are 4 unique regions and 4 unique quarters. The pivot table shows the sum of revenue for each possible combination. Having the data in this structure, can make it easier to read.

![][1]

[1]: images/pt.png

## Creating a simple pivot table in pandas - four components
There are four components to a basic pivot table in pandas.
* Two grouping columns: one is column(midwest, northest, south..) and the other is row (Q1-Q4)
* One aggregating column
* One aggregating function

In the example above, the two grouping columns are `Region` and `Qtr`. The aggregating column is `Revenue` and the aggregating function is `sum`.

# First pivot table - find the average salary by race and gender
After reading in the data, let's identify the components of this pivot table.
* Grouping columns - race and gender
* Aggregating column - salary
* Aggregating function - mean

In [1]:
import pandas as pd
emp = pd.read_csv('../data/employee.csv')
emp.head()

,title,dept,salary,race,gender,hire_date,job_date
0,ASSISTANT DIRECTOR (EX LVL),Municipal Courts Department,121862.0,Hispanic,Female,2006-06-12,2012-10-13
1,LIBRARY ASSISTANT,Library,26125.0,Hispanic,Female,2000-07-19,2010-09-18
2,POLICE OFFICER,Houston Police Department-HPD,45279.0,White,Male,2015-02-03,2015-02-03
3,ENGINEER/OPERATOR,Houston Fire Department (HFD),63166.0,White,Male,1982-02-08,1991-05-25
4,ELECTRICIAN,General Services Department,56347.0,White,Male,1989-06-19,1994-10-22


## Mapping the components of the pivot table to the parameters of the `pivot_table` method
The `pivot_table` method creates pivot tables for us in pandas. To use a pivot table, we set the `index`, `columns`, `values`, and `aggfunc` parameters. Each parameter will take on the following component of the pivot table.

* `index` - grouping column
* `columns` - grouping column
* `values` - aggregating column
* `aggfunc` - aggregating function (defaulted to the mean)

In [2]:
emp.pivot_table(index='race', columns='gender', 
                values='salary', aggfunc='mean')

gender,Female,Male
race,,
Asian,63226.300000,61033.906667
Black,48915.421233,51082.074074
Hispanic,46503.316176,54782.819018
Native American,60238.800000,60305.400000
Other,63785.000000,38771.000000
White,66793.352941,63940.388119


## Trick to reduce noise in dataframe - use `astype('int')`
The above dataframe has lots of excess decimal values that are completely useless in this result. Changing the data type of the columns to be integer instead of float will eliminate the noisy decimals. We do this by using the `astype` method, which takes a string of the new data type you would like to enforce on you data.

In [3]:
# much cleaner
emp.pivot_table(index='race', columns='gender',  # can replace 'race' with a list 
                values='salary', aggfunc='mean').astype('int') # force float to int

gender,Female,Male
race,,
Asian,63226,61033
Black,48915,51082
Hispanic,46503,54782
Native American,60238,60305
Other,63785,38771
White,66793,63940


### Easily compare female vs male salary
It is now trivial to compare female and male salaries for every race.

## Comparison to a group by
Since we already know the grouping columns, aggregating columns, and the aggregating functions, we should have no problem using a groupby.

In [5]:
emp.groupby(['race', 'gender']).agg({'salary': 'mean'}).astype('int') 
#same as above, but pivot looks nicer

salary
race            gender        
Asian           Female   63226
                Male     61033
Black           Female   48915
                Male     51082
Hispanic        Female   46503
                Male     54782
Native American Female   60238
                Male     60305
Other           Female   63785
                Male     38771
White           Female   66793
                Male     63940

## Data is more difficult to make comparisons
This groupby has produced the exact data as the pivot tabel but in a different format. Having all of our data in a vertical column makes it difficult to make comparisons.

## Wide vs long data
Pivot tables produce **wide** data meaning that it will often result in data that is easier to read and make decisions with. The `groupby` method returns **long** data that takes a bit more effort in making a comparison.

## All aggregation strings are available for `pivot_table`
All the aggregation strings ('min', 'max', 'mean', etc...) are available to a `pivot_table` in the same way there were with groupby. A groupby and a pivot table are essentially the same, but with a different output.

## The default aggregation is `mean`
By default, `pivot_table` takes the `mean` of each group.

## Using a different aggregating function
Use any valid aggregation string. Here we find the max salary.

In [4]:
emp.pivot_table(index='race', columns='gender', 
                values='salary', aggfunc='max').astype('int')

gender,Female,Male
race,,
Asian,130416,163228
Black,150416,275000
Hispanic,126115,165216
Native American,98536,81239
Other,63785,38771
White,178331,210588


## Where is the 'pivoting'?
In Excel, you can pivot the table easily by dragging columns to different into different boxes easily. With pandas, you'll have to write a bit of code to do this. Let's pivot the table by putting gender along the index and race along the columns.

In [11]:
emp.pivot_table(index='race', columns=['gender', 'dept'], 
                values='salary', aggfunc='max') 
# female on the left and male on the right

gender                              Female                           \
dept            Admn. & Regulatory Affairs City Controller's Office   
race                                                                  
Asian                             130416.0                  59077.0   
Black                              72741.0                  57054.0   
Hispanic                           47341.0                  64251.0   
Native American                        NaN                      NaN   
Other                                  NaN                      NaN   
White                              62129.0                      NaN   

gender                                                     \
dept            City Council Convention and Entertainment   
race                                                        
Asian                    NaN                          NaN   
Black               100000.0                          NaN   
Hispanic             35837.0                      38397.0   
Native American          NaN                          NaN   
Other                    NaN                          NaN   
White                77000.0                          NaN   

gender                                                \
dept            Dept of Neighborhoods (DON)  Finance   
race                                                   
Asian                                   NaN      NaN   
Black                               64424.0  81458.0   
Hispanic                            54829.0  95773.0   
Native American                         NaN      NaN   
Other                                   NaN      NaN   
White                               65832.0      NaN   

gender                                                                   \
dept            Fleet Management Department General Services Department   
race                                                                      
Asian                                   NaN                         NaN   
Black                               56940.0                         NaN   
Hispanic                                NaN                         NaN   
Native American                         NaN                         NaN   
Other                                   NaN                         NaN   
White                                   NaN                     72932.0   

gender                                                               \
dept            Health & Human Services Housing and Community Devp.   
race                                                                  
Asian                           94149.0                     70832.0   
Black                          103270.0                     67643.0   
Hispanic                        65589.0                     95791.0   
Native American                 58855.0                     98536.0   
Other                           63785.0                         NaN   
White                          100791.0                     78853.0   

gender                   ...                                    Male  \
dept                     ...           Houston Police Department-HPD   
race                     ...                                           
Asian                    ...                                163228.0   
Black                    ...                                104455.0   
Hispanic                 ...                                 91181.0   
Native American          ...                                 81239.0   
Other                    ...                                     NaN   
White                    ...                                199596.0   

gender                                                           \
dept            Human Resources Dept. Legal Department  Library   
race                                                              
Asian                             NaN          84107.0  31034.0   
Black                         53970.0         275000.0  60840.0   
Hispa

In [8]:
emp.pivot_table(index='race', columns='gender', 
                values='salary', aggfunc='max').astype('int')
# pivot is for display results, groupby more for analysis

gender,Female,Male
race,,
Asian,130416,163228
Black,150416,275000
Hispanic,126115,165216
Native American,98536,81239
Other,63785,38771
White,178331,210588


# The unique values of each grouping column for the labels
Notice that the labels for each of the index and columns of a pivot table come from the unique values of the grouping columns. The intersection of each label is where the aggregated data appears.

# Practice making pivot tables with employee data
* Make several pivot tables of information that you might find useful
* You can use multiple grouping columns, aggregating columns, and aggregating functions, but the result will be much more difficult to interpret.
* Create the experience column to get another numeric column

In [7]:
emp.head()

,title,dept,salary,race,gender,hire_date,job_date
0,ASSISTANT DIRECTOR (EX LVL),Municipal Courts Department,121862.0,Hispanic,Female,2006-06-12,2012-10-13
1,LIBRARY ASSISTANT,Library,26125.0,Hispanic,Female,2000-07-19,2010-09-18
2,POLICE OFFICER,Houston Police Department-HPD,45279.0,White,Male,2015-02-03,2015-02-03
3,ENGINEER/OPERATOR,Houston Fire Department (HFD),63166.0,White,Male,1982-02-08,1991-05-25
4,ELECTRICIAN,General Services Department,56347.0,White,Male,1989-06-19,1994-10-22


# Find a dataset on kaggle on make pivot tables with it

Navigate to the [Kaggle datasets page][1] and filter for smaller datasets. Choose one that interests you (use the search box for help). Download the data and put it in the data folder of this repository and do some analysis with it below.

[1]: https://www.kaggle.com/datasets